In [2]:
import django_initializer
from telemetry.models import FastLap, Lap
from telemetry.analyzer import Analyzer
from telemetry.influx import Influx
import plotly.io as pio

pio.renderers.default = "svg"  # comment this line to use interactive plots

import pandas as pd
from telemetry.visualizer import *

%load_ext autoreload
%autoreload 2

analyzer = Analyzer()
influx = Influx()

2023-06-04 18:06:45,064 DEBUG Influx: Connected to https://telemetry.b4mad.racing/


In [6]:
# Automobilista 2 - SpaFrancorchamps:Spa_Francorchamps_2022 - Mercedes-Benz CLK LM
# iRacing / richmond / Gen 4 Chevrolet Impala
kwargs = {
    "track__game__name": "iRacing",
    # "track__name": "fuji nochicane",
    # "track__name": "longbeach",
    "track__name": "oschersleben gp",
    # "track__name": "oran gp",
    # "track__name": "nurburgring combined",
    # "track__name": "richmond",
    "car__name": "Ferrari 488 GT3 Evo 2020",
    # "car__name": "Gen 4 Chevrolet Impala",
}
laps = Lap.objects.filter(**kwargs)
laps.order_by("time")
laps.first().time

144.4143